# Data Exploration
## Purpose
State the purpose of the notebook.
## Methodology
Quickly describle assumptions and processing steps.
## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:

- Todo 1;
- Todo 2;

## Results
Describe and comment the most important results.

# Setup
## Library import
We import all the required P
ython libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
import rasterio
import regionmask
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Utils

**set_lat_lon_attrs**

In [ ]:
def set_lat_lon_attrs(ds):
    """ Set CF latitude and longitude attributes"""
    ds["lon"] = ds.lon.assign_attrs({
      'axis' : 'X',
       'long_name' : 'longitude',
        'standard_name' : 'longitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_east',
            'valid_max' : 360.0,
             'valid_min' : -180.0
             })
    ds["lat"] = ds.lat.assign_attrs({
      'axis' : 'Y',
       'long_name' : 'latitude',
        'standard_name' : 'latitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_north',
            'valid_max' : 90.0,
             'valid_min' : -90.0
             })
    return ds


**create_ds_mask**

In [ ]:
def create_ds_mask(df, ds, name, lon_name='lon', lat_name='lat'):
    # Create index column
    if 'index' not in df:
        df = df.reset_index(drop=True).reset_index()
    
    # Get mean ds cell area (in degrees) 
    mean_y_size = np.diff(ds.lat.values).mean()
    #print(mean_y_size)
    mean_x_size = np.diff(ds.lat.values).mean()
    #print(mean_x_size)
    mean_area = mean_y_size * mean_x_size
    print(f"The mean ds cell area is {np.round(mean_area, 6)} deg.\n")
    
    # Clip gdf to bounding box of ds
    xmin = ds.lon.min().values.tolist()
    xmax = ds.lon.max().values.tolist()
    ymin = ds.lat.min().values.tolist()
    ymax = ds.lat.max().values.tolist()
    df = df.cx[xmin:xmax, ymin:ymax]
    
    
    # Add area of geoms to gdf
    df = df.assign(area = df.area)
    df = df.assign(area_is_gt_cell = df['area'] > mean_area)
    print(f"Clipped gdf to dataset bounds, giving {len(df['index'])} potential geometries, of which {df['area_is_gt_cell'].sum()} are large enough.\n")
    
    print("Geometries smaller than mean cell size:")
    print(df.loc[df['area_is_gt_cell'] == False, ['index']])
    print("\n")

    # Extract indexes and geoms that are large enough!
    id_ints = df.loc[df['area_is_gt_cell'] == True, 'index'].values
    geoms = df.loc[df['area_is_gt_cell'] == True, 'geometry'].values
    
    print(f'Number of indexes: {len(id_ints)}')
    print(f'Number of geoms: {len(geoms)}')


    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      outlines = geoms)\
      .mask(ds, lon_name=lon_name, lat_name=lat_name)\
      .rename(name)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    
    print(f'Number of ints in mask: {len(id_ints_mask)}')
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask = set_lat_lon_attrs(da_mask)
    return da_mask

# Data import

## Political boundaries for Spain 
**[Data source](https://gadm.org/download_country_v3.html)**

GADM data (version 3.6).

In [ ]:
# Read Political boundaries:
for level in range(3):
    if level == 0:
        spain = gpd.read_file(f'../../datasets/raw/gadm36_ESP_shp/gadm36_ESP_{str(level)}.shp')
        spain['level'] = level
    else:
        gdf = gpd.read_file(f'../../datasets/raw/gadm36_ESP_shp/gadm36_ESP_{str(level)}.shp')
        gdf['level'] = level
        spain = pd.concat([spain, gdf])

In [ ]:
spain

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
spain[spain['level'] == 2].plot(ax=ax)

## Fire danger indicators for Europe 
**[Data source](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-tourism-fire-danger-indicators?tab=overview)**

The dataset presents projections of fire danger indicators for Europe based upon the Canadian Fire Weather Index System (FWI) under future climate conditions. The FWI is a meteorologically based index used worldwide to estimate the fire danger and is implemented in the Global ECMWF Fire Forecasting model (GEFF).

**Variables:**
- **Seasonal fire weather index:** 
The mean fire weather index value over the European fire season (June-September). This is calculated as the sum of the daily fire weather index over the European fire season divided by the total number of days within this date range. The higher the index value, the more favorable the meteorological conditions to trigger a wildfire are.

**Read data**

In [ ]:
data_dir = '../../datasets/raw/seasonal-fire-weather-index/'
for n, file in enumerate(os.listdir(data_dir)):
    # convert to Dataset and concatenate by time
    if n == 0:
        ds_fire = xr.open_dataset(data_dir+file, engine="netcdf4")
    else:
        ds = xr.open_dataset(data_dir+file, engine="netcdf4")
        ds_fire = xr.concat([ds_fire, ds], dim='time')

In [ ]:
ds_fire

**Multidimensional Coordinates**

The data will derive from a numerical model in which the poles of the model's coordinate system (`logical coordinates`) differ from the earth's true poles (`physical coordinates`). This is typically done when running limited area models, in order to keep the poles as far away as possible from the area that is being modelled. This allows the model's resolution to be roughly uniform over the model domain, as the coordinate system is then approximately cartesian and avoids issues where the meridians converge close to the poles.

In our dataset, the `logical coordinates` are `rlon` and `rlat`, while the physical coordinates are `lon` and `lat`, which represent the latitudes and longitude of the data.

In [ ]:
print(ds_fire.rlon.attrs)
print(ds_fire.rlat.attrs)

In [ ]:
print(ds_fire.lon.attrs)
print(ds_fire.lat.attrs)

**Display data**

Xarray provides [several ways](http://xarray.pydata.org/en/stable/examples/multidimensional-coords.html) to plot and analyze such datasets.

If we try to plot the data variable `fwi-mean-jjas`, by default we get the logical coordinates.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,10))
ds_fire['fwi-mean-jjas'].isel(time=0).plot(ax=axs[0])
ds_fire['fwi-mean-jjas'].isel(time=77).plot(ax=axs[1])

In order to visualize the data on a conventional latitude-longitude grid, we can take advantage of xarray’s ability to apply [cartopy](http://scitools.org.uk/cartopy/index.html) map projections.

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ds_fire['fwi-mean-jjas'].isel(time=0).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False)
ax.coastlines()
ax.set_ylim([0,90]);
ax.set_xlim([-30,70]);

### Rasterize vector data

Create the data mask by rasterizing the vector data

In [ ]:
gdf = spain[spain['level'] == 0].copy()
da_mask = create_ds_mask(gdf, ds_fire, name='mask', lon_name='lon', lat_name='lat')

Add mask as a new variable into the xarray.Dataset

In [ ]:
ds_fire['mask'] = da_mask
ds_fire

**Display values for Spain**

In [ ]:
index = 0
xmin, ymin, xmax, ymax = gdf.iloc[index]['geometry'].bounds
ds_fire_spain = ds_fire['fwi-mean-jjas'].where(ds_fire.mask == index).copy()

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ds_fire_spain.isel(time=77).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False)
ax.coastlines()
ax.set_ylim([ymin-1, ymax+1]);
ax.set_xlim([xmin-1, xmax+1]);

**Compute mean value over time**

In [ ]:
plt.plot(ds_fire_spain.where(ds_fire.mask == index).mean(['rlon', 'rlat']).values)